In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/banana-ripeness-classification-dataset/Banana Ripeness Classification Dataset/valid/unripe/musa-acuminata-freshunripe-1e57db85-2653-11ec-802f-d8c4975e38aa_jpg.rf.133256b07ad985a5d5bce153aed4caf6.jpg
/kaggle/input/banana-ripeness-classification-dataset/Banana Ripeness Classification Dataset/valid/unripe/musa-acuminata-freshunripe-296b3a0b-2653-11ec-b6fc-d8c4975e38aa_jpg.rf.0c694ae3284c438bc7ec44b13ccd4a3e.jpg
/kaggle/input/banana-ripeness-classification-dataset/Banana Ripeness Classification Dataset/valid/unripe/musa-acuminata-unripe-56599d55-2653-11ec-89c5-d8c4975e38aa-Copy_jpg.rf.a8fd88f0884c02539159bb2f3d86fe83.jpg
/kaggle/input/banana-ripeness-classification-dataset/Banana Ripeness Classification Dataset/valid/unripe/musa-acuminata-freshunripe-341dc6ef-2653-11ec-9d7a-d8c4975e38aa_jpg.rf.f65745da86baffb3a112e1592848b53e.jpg
/kaggle/input/banana-ripeness-classification-dataset/Banana Ripeness Classification Dataset/valid/unripe/musa-acuminata-unripe-877ebe39-1d0a-11ec-ae

In [2]:

import tensorflow as tf
#import keras.preprocessing.image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout , BatchNormalization
from keras.optimizers import Adam, Adamax
from keras import regularizers
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report

training = '/kaggle/input/banana-ripeness-classification-dataset/Banana Ripeness Classification Dataset/train'
validation = '/kaggle/input/banana-ripeness-classification-dataset/Banana Ripeness Classification Dataset/valid'
test = '/kaggle/input/banana-ripeness-classification-dataset/Banana Ripeness Classification Dataset/test'


train_dataset = tf.keras.utils.image_dataset_from_directory(
    training,
    labels='inferred',             # Automatically infers labels from folder names
    label_mode='categorical',              # Options: 'int', 'categorical', 'binary', or None
    batch_size=32,                 # Number of images per batch
    image_size=(256, 256),         # Resize images to this size
    shuffle=True,                  # Shuffle the dataset
    seed=123                       # Random seed for reproducibility
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    validation,
    labels='inferred',             
    label_mode='categorical',             
    batch_size=32,                 
    image_size=(256, 256),         
    shuffle=True,                  
    seed=123)                     

print(train_dataset.class_names)


2025-05-19 15:36:51.669435: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747669011.856545      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747669011.910405      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 11793 files belonging to 4 classes.


I0000 00:00:1747669027.741893      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1747669027.742655      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Found 1123 files belonging to 4 classes.
['overripe', 'ripe', 'rotten', 'unripe']


In [3]:
num_classes = 4

model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(4, activation = 'softmax')
    ]
)

In [4]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
  metrics=['accuracy'])

In [5]:
cnn_model = model.fit(
      train_dataset,
      validation_data=val_dataset,
      epochs=30
    )

Epoch 1/30


I0000 00:00:1747669034.261927      69 service.cc:148] XLA service 0x7dd4200052b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747669034.262522      69 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1747669034.262543      69 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1747669034.590775      69 cuda_dnn.cc:529] Loaded cuDNN version 90300


  3/369 ━━━━━━━━━━━━━━━━━━━━ 15s 41ms/step - accuracy: 0.2917 - loss: 1.7516

I0000 00:00:1747669039.026409      69 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


369/369 ━━━━━━━━━━━━━━━━━━━━ 30s 55ms/step - accuracy: 0.6543 - loss: 0.8251 - val_accuracy: 0.9101 - val_loss: 0.2466
Epoch 2/30
369/369 ━━━━━━━━━━━━━━━━━━━━ 16s 43ms/step - accuracy: 0.8890 - loss: 0.3047 - val_accuracy: 0.8433 - val_loss: 0.4130
Epoch 3/30
369/369 ━━━━━━━━━━━━━━━━━━━━ 16s 43ms/step - accuracy: 0.9144 - loss: 0.2331 - val_accuracy: 0.8557 - val_loss: 0.3630
Epoch 4/30
369/369 ━━━━━━━━━━━━━━━━━━━━ 16s 43ms/step - accuracy: 0.9281 - loss: 0.1781 - val_accuracy: 0.9323 - val_loss: 0.2122
Epoch 5/30
369/369 ━━━━━━━━━━━━━━━━━━━━ 16s 43ms/step - accuracy: 0.9518 - loss: 0.1373 - val_accuracy: 0.9555 - val_loss: 0.1670
Epoch 6/30
369/369 ━━━━━━━━━━━━━━━━━━━━ 16s 44ms/step - accuracy: 0.9646 - loss: 0.1014 - val_accuracy: 0.9412 - val_loss: 0.2295
Epoch 7/30
369/369 ━━━━━━━━━━━━━━━━━━━━ 16s 44ms/step - accuracy: 0.9730 - loss: 0.0716 - val_accuracy: 0.9207 - val_loss: 0.3355
Epoch 8/30
369/369 ━━━━━━━━━━━━━━━━━━━━ 16s 43ms/step - accuracy: 0.9772 - loss: 0.0699 - val_accurac

In [6]:
test_dataset = tf.keras.utils.image_dataset_from_directory(
    test,
    labels='inferred',             
    label_mode='categorical',             
    batch_size=32,                 
    image_size=(256, 256),         
    shuffle=True,                  
    seed=123)                     

Found 562 files belonging to 4 classes.


In [7]:
y_true = []
y_pred = []

for images, labels in test_dataset:
    predictions = model.predict(images, verbose=0)

    # Handle one-hot encoded labels or integer labels
    if len(labels.shape) > 1 and labels.shape[1] > 1:
        labels = tf.argmax(labels, axis=1)  # Convert one-hot to class index

    preds = tf.argmax(predictions, axis=1)

    y_true.extend(labels.numpy())
    y_pred.extend(preds.numpy())

# Generate confusion matrix
cm = confusion_matrix(y_true, y_pred)

In [8]:
print(cm)

from sklearn.metrics import classification_report
print(classification_report(y_true,y_pred,target_names = ['overripe','ripe','rotten','underripe']))

[[112   1   0   0]
 [  6 147   1   0]
 [ 15   6 161   3]
 [  0   2   2 106]]
              precision    recall  f1-score   support

    overripe       0.84      0.99      0.91       113
        ripe       0.94      0.95      0.95       154
      rotten       0.98      0.87      0.92       185
   underripe       0.97      0.96      0.97       110

    accuracy                           0.94       562
   macro avg       0.93      0.94      0.94       562
weighted avg       0.94      0.94      0.94       562

